# Data clean

In [56]:
import numpy as np 
import pandas as pd
import datetime
import plotly.express as px
pre_data = pd.read_csv('/Users/camille/repo/projet_perso/queer_project/Data/tdor_export.csv')
pre_data["Date"] = pd.to_datetime(pre_data["Date"])
pre_data["year"] = pre_data["Date"].dt.year
pre_data = pre_data[pre_data['year'] >= 1999]
pre_data["nb_victims_year"] = pre_data.groupby('year').year.transform("count")
pre_data["nb_victims_Category"] = pre_data.groupby('Category').Category.transform("count")
pre_data["nb_victims_country"] = pre_data.groupby('Country').Country.transform("count")
pre_data["nb_victims_country_year"] = pre_data.groupby(['Country', 'year']).year.transform("count")
pre_data["nb_victims_category_country_year"] = pre_data.groupby(['Country', 'year', 'Category']).year.transform("count")
pre_data["nb_victims_Category_year"] = pre_data.groupby(['Category', 'year']).year.transform("count")
pre_data["nb_victims_cause_year"] = pre_data.groupby(['Cause of death', 'year']).year.transform("count")
pre_data["Birthdate"] = pd.to_datetime(pre_data["Birthdate"])
pre_data["year_birthdays"] = pre_data["Birthdate"].dt.year.astype('Int64')
pre_data["age"] = pre_data["year"] - pre_data["year_birthdays"] 
tdor_data = pre_data.drop(["Age", "Photo","Photo source","Thumbnail","Tweet","Permalink","QR code","Description", "TDoR list ref"], axis=1)
tdor_data["Category"]= tdor_data.Category.str.capitalize()
tdor_data.rename(columns = {'Cause of death':'Cause_of_the_death'}, inplace = True)
tdor_data.rename(columns = {'Country Code':'Country_Code'}, inplace = True)
tdor_data['Country_Code'] = tdor_data.Country_Code.str.capitalize()
tdor_data["Cause_of_the_death"] = tdor_data.Cause_of_the_death.str.capitalize()

tdor_data['percentage_category'] = ((tdor_data["nb_victims_Category_year"] / 
                      tdor_data["nb_victims_year"]) * 100).round(1)


In [57]:
df_pop= pd.read_csv('/Users/camille/repo/projet_perso/queer_project/Data/population_world.csv', index_col=0)
df_pop.reset_index(inplace=True)
df_pop.drop(['SortOrder',
 'LocID',
 'Notes',
 'ISO3_code',
 'ISO2_code',
 'SDMX_code',
 'LocTypeID',
 'LocTypeName',
 'ParentID', 
 'VarID',
 'MidPeriod',
 'PopMale',
 'PopFemale', 
 'PopDensity'], axis = 1 , inplace=True)  
df_pop = df_pop[df_pop['Variant'] == 'Medium']
df_pop["PopTotal"] = (df_pop["PopTotal"]* 1000).astype(int)
df_pop.drop(['Variant'], axis = 1 , inplace=True)
df_pop.rename(columns={'Location':'Country', 'Time':'Year', 'PopTotal':'Population'}, inplace=True)
df_pop['Country'] = df_pop['Country'].replace(['United States of America'], 'USA')


/var/folders/vh/yn_93j315gn6jdvz7f2d73_m0000gn/T/ipykernel_43548/111625705.py:1: DtypeWarning: Columns (2,3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pop= pd.read_csv('/Users/camille/repo/projet_perso/queer_project/Data/population_world.csv', index_col=0)


In [58]:
tdor_merge_pop = df_pop.merge( tdor_data, left_on=['Year','Country'], right_on=['year','Country'], how='right')

In [59]:
tdor_Violence = tdor_merge_pop[tdor_merge_pop['Category'] == 'Violence']
tdor_Suicide = tdor_merge_pop[tdor_merge_pop['Category'] == 'Suicide']
tdor_Medical= tdor_merge_pop[tdor_merge_pop['Category'] == 'Medical']
tdor_Custodial= tdor_merge_pop[tdor_merge_pop['Category'] == 'Custodial']
tdor_Uncategorised= tdor_merge_pop[tdor_merge_pop['Category'] == 'Uncategorised']

In [60]:
tdor_merge_pop["ratio"] = (tdor_merge_pop["nb_victims_country_year"]/tdor_merge_pop["Population"])*100000
tdor_merge_pop["ratio"] = tdor_merge_pop["ratio"].round(3)
tdor_Medical ["ratio"] = (tdor_Medical ["nb_victims_category_country_year"]/tdor_Medical ["Population"])*100000
tdor_Medical ["ratio"] = tdor_Medical ["ratio"].round(3)
tdor_Violence["ratio"] = (tdor_Violence["nb_victims_category_country_year"]/tdor_Violence["Population"])*100000
tdor_Violence["ratio"] = tdor_Violence["ratio"].round(3)
tdor_Custodial ["ratio"] = (tdor_Custodial ["nb_victims_category_country_year"]/tdor_Custodial ["Population"])*100000
tdor_Custodial ["ratio"] = tdor_Custodial ["ratio"].round(3)
tdor_Uncategorised ["ratio"] = (tdor_Uncategorised ["nb_victims_category_country_year"]/tdor_Uncategorised ["Population"])*100000
tdor_Uncategorised ["ratio"] = tdor_Uncategorised ["ratio"].round(3)

/var/folders/vh/yn_93j315gn6jdvz7f2d73_m0000gn/T/ipykernel_43548/4166768537.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdor_Medical ["ratio"] = (tdor_Medical ["nb_victims_category_country_year"]/tdor_Medical ["Population"])*100000
/var/folders/vh/yn_93j315gn6jdvz7f2d73_m0000gn/T/ipykernel_43548/4166768537.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdor_Medical ["ratio"] = tdor_Medical ["ratio"].round(3)
/var/folders/vh/yn_93j315gn6jdvz7f2d73_m0000gn/T/ipykernel_43548/4166768537.py:5: Setti

In [62]:
tdor_merge_pop["ratio"] = (tdor_merge_pop["nb_victims_country_year"]/tdor_merge_pop["Population"])*100000
tdor_merge_pop["ratio"] = tdor_merge_pop["ratio"].round(3)
tdor_merge_pop.to_csv('tdor_data_all.csv')

In [63]:
tdor_merge_pop

,Country,Year,Population,Name,Birthdate,Date,Location,Country_Code,Latitude,Longitude,...,nb_victims_Category,nb_victims_country,nb_victims_country_year,nb_victims_category_country_year,nb_victims_Category_year,nb_victims_cause_year,year_birthdays,age,percentage_category,ratio
0,USA,1999.0,279181581.0,Lauryn Paige Fuller,NaT,1999-01-09,"Austin, Texas",Us,30.264979,-97.746598,...,3889,609,14,13,14,1,<NA>,<NA>,93.3,0.005
1,USA,1999.0,279181581.0,Robert Eads,1945-12-18,1999-01-17,"Atlanta, Georgia",Us,33.748547,-84.391502,...,92,609,14,1,1,1,1945,54,6.7,0.005
2,USA,1999.0,279181581.0,S. D. Garcia,1964-08-03,1999-02-06,"Houston, Texas",Us,29.755008,-95.390479,...,3889,609,14,13,14,4,1964,35,93.3,0.005
3,USA,1999.0,279181581.0,Chris Muzett (Eddie Matthews),NaT,1999-02-20,"Detroit, Michigan",Us,42.332916,-83.047853,...,3889,609,14,13,14,2,<NA>,<NA>,93.3,0.005
4,USA,1999.0,279181581.0,Name Unknown,NaT,1999-02-25,"Houston, Texas",Us,29.760803,-95.369506,...,3889,609,14,13,14,4,<NA>,<NA>,93.3,0.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4683,USA,2022.0,338289857.0,Mar’Quis ‘MJ’ Jackson,1989-12-12,2022-12-14,"Philadelphia, Pennsylvania",Us,40.011247,-75.180731,...,3889,609,56,39,282,24,1989,33,77.5,0.017
4684,Mexico,2022.0,127504125.0,Name Unknown,NaT,2022-12-16,"Tijuana, Baja California",Mx,32.538830,-117.043072,...,3889,629,58,46,282,124,<NA>,<NA>,77.5,0.045
4685,Argentina,2022.0,45510318.0,Ariana Diaz Chavez,NaT,2022-12-16,"La Plata, Buenos Aires",Ar,-34.920561,-57.944332,...,3889,136,11,9,282,13,<NA>,<NA>,77.5,0.024
4686,USA,2022.0,338289857.0,Caelee Love-Light,1995-05-17,2022-12-17,"Phoenix, Arizona",Us,33.378008,-112.107799,...,3889,609,56,39,282,124,1995,27,77.5,0.017


In [61]:
tdor_Violence.to_csv('tdor_Violence.csv')
tdor_Suicide.to_csv('tdor_suicide.csv')
tdor_Custodial.to_csv('tdor_Custodial.csv')
tdor_Medical.to_csv('tdor_Medical.csv')
tdor_Uncategorised.to_csv('tdor_Uncategorised.csv')

In [64]:
test=tdor_data.sort_values(['nb_victims_Category'],ascending=False).groupby('Category').head(3).drop_duplicates('Category', keep='last')

In [65]:
test

,Name,Birthdate,Date,Location,Country,Country_Code,Latitude,Longitude,Category,Cause_of_the_death,...,nb_victims_year,nb_victims_Category,nb_victims_country,nb_victims_country_year,nb_victims_category_country_year,nb_victims_Category_year,nb_victims_cause_year,year_birthdays,age,percentage_category
3081,Linda,NaT,2018-07-26,Estado de Mexico,Mexico,Mx,19.483945,-99.689972,Violence,Suffocated,...,392,3889,629,66,62,339,1,<NA>,<NA>,86.5
455,Name Unknown,NaT,2008-12-31,NaN,Brazil,Br,-10.861639,-53.104038,Uncategorised,Not reported,...,53,413,1818,14,12,16,16,<NA>,<NA>,30.2
3304,Flávia Santana,NaT,2019-03-08,"Anápolis, Goiás",Brazil,Br,-16.333283,-48.952576,Suicide,Possible suicide,...,375,257,1818,139,20,47,6,<NA>,<NA>,12.5
3785,Aghatha Lima (Maria Aghata Jeremias de Souza L...,NaT,2020-05-17,"Ribeirão Preto, São Paulo",Brazil,Br,-21.178333,-47.806667,Medical,Cosmetic filler complications,...,463,92,1818,190,9,24,3,<NA>,<NA>,5.2
4592,Name Unknown,NaT,2022-02-25,Hamburg,Germany,De,53.626088,10.019663,Custodial,Suicide,...,364,37,6,2,1,6,45,<NA>,<NA>,1.6


# Graph